In [2]:
import string

In [5]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [3]:
#|hide
from nbdev.showdoc import show_doc

In [4]:
#|hide
import project_2025 as proj
from project_2025 import const, log, utils, tools
import adu_proj.utils as adutils

In [5]:
import fitz  # PyMuPDF
import re
import json
import unicodedata
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
import numpy as np
import pdfplumber
import PyPDF2
#|export
from openai import OpenAI
from dotenv import load_dotenv
from fuzzywuzzy import fuzz
import os
import json
import lancedb
import pdfplumber
import re
# import asyncio
import yaml
import pandas as pd
# from titlecase import titlecase
from tqdm import tqdm
from lancedb.pydantic import LanceModel, Vector
from copy import deepcopy

/Users/seangreaves/miniconda3/envs/project_2025/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [1]:
import json
from typing import List
from sentence_transformers import SentenceTransformer
import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry
import os
import shutil

/Users/seangreaves/miniconda3/envs/project_2025/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
with open('../data/index_items.json', 'r') as file:
    entities = json.load(file)

In [9]:
lancedb_path = "../.lancedb"

Delete the .lancedb folder before running the code:

In [ ]:
if os.path.exists(lancedb_path) and os.path.isdir(lancedb_path):
    # Remove the directory and all its contents
    shutil.rmtree(lancedb_path)
    print(f"The directory '{lancedb_path}' has been removed.")
else:
    print(f"The directory '{lancedb_path}' does not exist.")

In [3]:
db = lancedb.connect(lancedb_path)
title_model = get_registry().get("sentence-transformers").create(name="BAAI/bge-small-en-v1.5", device="cpu")
summary_model = get_registry().get("sentence-transformers").create(name="BAAI/bge-small-en-v1.5", device="cpu")

In [4]:
class Words(LanceModel):
    title: str = title_model.SourceField()
    title_vector: Vector(title_model.ndims()) = title_model.VectorField() 
    summary: str = summary_model.SourceField()
    summary_vector: Vector(summary_model.ndims()) = summary_model.VectorField() 
    section: str
    page: int
    wiki: bool
    title_index: int
    instance_of: List[str]
    wikipedia_url: str
    wikidata_id: str

In [5]:
entity_table = db.create_table("entities", schema=Words)
entity_table.add(
    entities
)

### Load

In [ ]:
db = lancedb.connect(lancedb_path)

In [10]:
table = db.open_table("entities")